# Most Frequently Used next-app prediction

In [9]:
from torch import optim
from data.dataloader import DatasetLoader
from torch.utils.data import DataLoader
import torch
from models.BiGRU import BiGRU
from torch import nn
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Dataset uitlezen

In [10]:
""" batch size 16 voor pre-trainen """
# dataset 1
# dataset = DatasetLoader("eventlog/phone_usage_cleaned.csv", seq_length=1)

# dataset 2
# dataset = DatasetLoader("eventlog/phone_usage_cleaned2.csv", seq_length=1)

# dataset 3
# dataset = DatasetLoader("eventlog/phone_usage_cleaned3.csv", seq_length=1)

removeapps = ["Screen on (locked)",
              "Screen off (locked)",
              "Screen on (unlocked)",
              "Screen off",
              "Samsung Experience Service",
              "Package installer",
              "System UI",
              "Customisation Service",
              "Configuration update",
              "EmergencyManagerService",
              "DeviceKeystring",
              "Samsung Keyboard",
              "HwModuleTest",
              "Device shutdown",
              "Device boot""com.android.systemui",
              "com.samsung.android.securitylogagent",
              "com.samsung.android.app.galaxyfinder",
              "com.samsung.android.incallui",
              "com.samsung.android.dialer",
              "com.sec.android.app.simsettingmgr",
              "com.samsung.android.app.telephonyui",
              "com.sec.android.wallpapercropper2",
              "com.samsung.android.kidsinstaller",
              "com.sec.android.app.soundalive",
              "com.samsung.android.setting.multisound",
              "com.wssyncmldm",
              "com.sec.android.preloadinstaller",
              "com.samsung.android.game.gametools",
              "com.android.settings.intelligence",
              "com.samsung.android.app.smartcapture",
              "com.samsung.android.visionintelligence",
              "com.samsung.android.MtpApplication",
              "com.android.captiveportallogin",
              "com.samsung.accessibility",
              "com.mgoogle.android.gms",
              "com.android.providers.media",
              "com.android.server.telecom",
              "com.diotek.sec.lookup.dictionary",
              "Permissions Controller",
              "com.samsung.app.newtrim",
              "S Suggest",
              "com.samsung.android.mdx.quickboard",
              "com.osp.app.signin",
              "Tool voor gegevensherstel",
              "com.samsung.android.bixby.agent",
              "DocumentsUI",
              "com.samsung.android.bluelightfilter",
              "android.instant_app",
              "Samsung Cloud Wear",
              "com.samsung.android.svoiceime",
              "com.samsung.android.rubin.app",
              "com.samsung.android.honeyboard",
              "Samsung Global Goals",
              "com.samsung.android.motionphoto.viewer",
              "com.samsung.android.net.wifi.wifiguider"""
              "com.aura.oobe.samsung.gl",
              "com.samsung.android.secsoundpicker",
              "com.samsung.android.service.tagservice",
              "com.samsung.android.arzone",
              "com.samsung.android.game.gamehome",
              "com.sec.android.app.vepreload",
              "Android",
              "com.samsung.android.smartmirroring",
              "Captive Portal",
              "com.android.systemui",
              "OnePlus Launcher",
              "com.oneplus.opbackup",
              "com.android.documentsui",
              "com.android.mms",
              "com.google.android.packageinstaller",
              "com.android.companiondevicemanager",
              "com.android.server.telecom",
              "com.android.captiveportallogin",
              "com.android.printspooler",
              "com.oneplus.healthcheck",
              "com.android.hotwordenrollment.okgoogle",
              "com.android.emergency",
              "Gboard: het Google-toetsenbord",
              "com.android.vpndialogs",
              "com.oneplus.security",
              "com.qualcomm.qti.networksetting",
              "com.android.settings.intelligence",
              "Device ID",
              "com.android.providers.downloads.ui",
              "com.android.bips",
              "com.apkpure.aegon",
              "com.android.nfc",
              "OnePlus Nord AR",
              "com.android.packageinstaller",
              "com.bluestacks.appmart",
              "com.uncube.account",
              "com.android.provision",
              "com.android.certinstaller",
              "EXT Services",
              "DocumentsUI",
              "Permissions Controller",
              "com.oneplus.wifiapsettings",
              "Telefoon van Google",
              "com.google.android.setupwizard",
              "com.oneplus.setupwizard",
              "Search Engine Selector",
              "Android",
              "com.android.stk",
              "Captive Portal",
              "com.android.systemui"]
dataset.clean(removeapps)

train_dataloader = DataLoader(dataset, batch_size=1)

## pre-train de MFU waarden voor 2000 eerste applicaties

In [11]:
appcount = np.zeros(shape=(200,), dtype=np.int) # index==app_index, value=keer_geopend
nr_opened_apps = 0

for prev_app_index, _ in train_dataloader:

    appcount[prev_app_index.item()] += 1

    nr_opened_apps+=1
    if nr_opened_apps==2000:
        break

print(appcount)
print("nultoken is voor padding")

[115  29 237 192 159 185  22 198 177  90   8  67   7  13  35  32   3   4
   1 192  36   3  54  14   2  16   1   2  32  12   7   1   4   4   1  16
   1   2  12   7   2   1   1   1   2   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
nultoken is voor padding


## Maak predicties en update MFU waarden voor de rest van de applicaties

In [12]:
nr_opened_apps = 0
running_accuracy_list = []

for _, next_app_index in train_dataloader:

    nr_opened_apps+=1
    if nr_opened_apps > 2000:
        mostfreq = appcount.argsort()[-5:][::-1]
        running_accuracy_list.append(next_app_index.item() in mostfreq)

        appcount[next_app_index.item()] += 1

## Accuracy uitschrijven naar comparison

In [13]:
# dataset 1
# AccuracyComparison = pd.read_csv("outputdata/accuracycomparison.csv", index_col=False)
# AccuracyComparison["MFU"] = running_accuracy_list
# AccuracyComparison.to_csv("outputdata/accuracycomparison.csv", index=False)
# AccuracyComparison.head()

# dataset 2
# AccuracyComparison = pd.read_csv("outputdata/accuracycomparison2.csv", index_col=False)
# AccuracyComparison["MFU"] = running_accuracy_list
# AccuracyComparison.to_csv("outputdata/accuracycomparison2.csv", index=False)
# AccuracyComparison.head()

# dataset 3
# AccuracyComparison = pd.read_csv("outputdata/accuracycomparison3.csv", index_col=False)
# AccuracyComparison["MFU"] = running_accuracy_list
# AccuracyComparison.to_csv("outputdata/accuracycomparison3.csv", index=False)
# AccuracyComparison.head()


,onehotlstm64,onehotlstm128,onehotgru64,onehotgru128,onehotbigru,MFU
0,True,True,True,True,True,True
1,True,True,True,True,True,True
2,True,True,True,True,True,False
3,True,True,True,True,True,False
4,False,False,True,True,False,True
